In [1]:
# Demo for yield prediction based on https://github.com/ADA-research/AutoML4HybridEarthScienceModels

import warnings
import numpy as np
from IPython.core.display import display
import geopandas as gpd
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit
import seaborn as sns
# import pixel processing functions
from yield_helper import create_geometry, getYieldwithoutBorders, create_dataset, flatten_time_series, filter_by_2std

sns.set_theme()
sns.set_style("darkgrid")

warnings.filterwarnings("ignore")

/tmp/ipykernel_34236/2108598746.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
# select the fields for prediction
fields = [
    'Baumacker', 'D8', 'Dichtlacker', 'Heindlacker', 'Heng', 'Holzacker',
    'Neulandsiedlung', 'Itzling2', 'Itzling5', 'Itzling6', 'Schluetterfabrik',
    'Thalhausen138', 'Thalhausen141', 'Voettingerfeld'
]

# fields =  ['Dichtlacker', 'Heindlacker', 'Heng',
#                            'Holzacker', 'Neulandsiedlung','Itzling5',
#                            'Itzling6', 'Schluetterfabrik', 'Thalhausen138', 'Voettingerfeld']

# test_fields = ['Baumacker', 'Itzling2', 'Thalhausen141']

# load summary for each field
field_summary = pd.read_excel(
    "../data/cropdata/Bavaria/yields/fields_summary.xlsx")
yields_df = pd.read_csv("../data/cropdata/Bavaria/yields/yields2018.csv")
print('Fields in yield data:', yields_df.Name.unique().tolist())

# select bands and other features
bands = ["B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"]
angles = ['solar_zenith', 'observer_zenith', 'relative_azimuth']
other_features = ["et0", "rain", "cum_rain"]
feature_cols = bands + other_features
target_col = "Ertr.masse (Nass)(tonne/ha)"

# Load shapefile
geo_df = gpd.read_file(
    '../data/cropdata/Bavaria/yields/FeldstueckeTUM/Feldstuecke_WGS84.shp')

yields_gdf = create_geometry(yields_df)

conversion = 1
# Apply the optimized function
yields_df = yields_gdf.groupby(['Name']).apply(
    lambda x: getYieldwithoutBorders(x, geo_df, conversion))
yields_df.reset_index(drop=True, inplace=True)

df = create_dataset(bands=bands, yields_df=yields_df, fields=fields)
out_df, feature_cols = flatten_time_series(df, feature_cols,
                                           "Ertr.masse (Nass)(tonne/ha)")

mean = out_df['Ertr.masse (Nass)(tonne/ha)'].mean()
std = out_df['Ertr.masse (Nass)(tonne/ha)'].std()

# out_df = filter_by_2std(mean, std,'Ertr.masse (Nass)(tonne/ha)', out_df )

Fields in yield data: ['Grafenfeld', 'Krohberg', 'Radarstation', 'Sieblerfeld', 'Striegelfeld', 'Baumacker', 'D2', 'D3', 'D4', 'D8', 'Dichtlacker', 'Feldhof1', 'Feldhof1a', 'Feldhof2', 'Heindlacker', 'Heng', 'Holzacker', 'Itzling2', 'Itzling4', 'Itzling5', 'Itzling6', 'Muehlacker', 'Neulandsiedlung', 'Schluetterfabrik', 'Thalhausen138', 'Thalhausen141', 'Viehhausen1', 'Viehhausen11', 'Viehhausen3', 'Viehhausen5', 'Voettingerfeld']


100%|██████████| 14/14 [01:10<00:00,  5.02s/it]


In [5]:
out_df.describe()

,B04_t-20,B05_t-20,B06_t-20,B07_t-20,B08_t-20,B8A_t-20,B09_t-20,B11_t-20,B12_t-20,et0_t-20,...,B07_t-0,B08_t-0,B8A_t-0,B09_t-0,B11_t-0,B12_t-0,et0_t-0,rain_t-0,cum_rain_t-0,Ertr.masse (Nass)(tonne/ha)
count,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4.637000e+03,...,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4637.000000,4.637000e+03,4.637000e+03,4.637000e+03,4637.000000
mean,0.000009,0.000012,0.000017,0.000018,0.000020,0.000020,0.000020,0.000019,0.000013,1.298619e+00,...,0.000019,0.000020,0.000021,0.000022,0.000025,0.000018,4.420460e+00,3.440000e-01,3.213420e+02,7.550732
std,0.000002,0.000003,0.000003,0.000003,0.000004,0.000003,0.000003,0.000003,0.000002,2.220686e-16,...,0.000008,0.000008,0.000009,0.000009,0.000012,0.000009,1.776548e-15,1.110343e-16,5.684955e-14,1.721437
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.298619e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.420460e+00,3.440000e-01,3.213420e+02,3.368000
25%,0.000007,0.000011,0.000015,0.000016,0.000017,0.000017,0.000018,0.000017,0.000012,1.298619e+00,...,0.000014,0.000015,0.000016,0.000017,0.000020,0.000014,4.420460e+00,3.440000e-01,3.213420e+02,6.274000
50%,0.000009,0.000013,0.000017,0.000018,0.000020,0.000019,0.000020,0.000019,0.000013,1.298619e+00,...,0.000018,0.000018,0.000020,0.000024,0.000022,0.000016,4.420460e+00,3.440000e-01,3.213420e+02,7.671000
75%,0.000010,0.000014,0.000019,0.000020,0.000022,0.000022,0.000022,0.000020,0.000015,1.298619e+00,...,0.000027,0.000028,0.000030,0.000030,0.000037,0.000026,4.420460e+00,3.440000e-01,3.213420e+02,8.872000
max,0.000016,0.000020,0.000032,0.000037,0.000035,0.000036,0.000038,0.000029,0.000022,1.298619e+00,...,0.000042,0.000043,0.000044,0.000045,0.000047,0.000034,4.420460e+00,3.440000e-01,3.213420e+02,11.400000


In [6]:
out_df[feature_cols]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(out_df[feature_cols].values)

scaler = preprocessing.StandardScaler()
scaler.fit(out_df[feature_cols].values)
x_scaled2 = scaler.transform(out_df[feature_cols].values)

test = pd.DataFrame(columns=feature_cols, data=x_scaled2)

# filter data
out_df = filter_by_2std(mean, std, 'Ertr.masse (Nass)(tonne/ha)', out_df)

In [7]:
# Train RF with pixels and apply
cv = ShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
rf = RandomForestRegressor(n_jobs=4, n_estimators=100)

results = cross_validate(rf,
                         X=out_df[feature_cols],
                         y=out_df[target_col],
                         cv=cv,
                         scoring=('r2', 'neg_mean_squared_error'),
                         return_train_score=True)

display("Mean train R2: {}, individual folds: {}".format(
    np.mean(results["train_r2"]), results["train_r2"]))
display("Mean test R2: {}, individual folds: {}".format(
    np.mean(results["test_r2"]), results["test_r2"]))

'Mean train R2: 0.9382038496354482, individual folds: [0.93754293 0.93758485 0.94111838 0.93707099 0.93770209]'

'Mean test R2: 0.5535455259341691, individual folds: [0.5632135  0.55860653 0.54544467 0.550136   0.55032693]'